In [39]:
import pandas as pd
import gensim
from gensim.corpora import Dictionary
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
from textblob import TextBlob
import re
from gensim import corpora
from gensim import models

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\annegru\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\annegru\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
uk_speed = pd.read_excel(r'C:UK_neg_speed.xlsx')
uk_reliability = pd.read_excel(r'UK_neg_reliability.xlsx')

uk_speed

In [7]:
uk_speed = uk_speed['Comment'].tolist()
uk_reliability= uk_reliability['Comment'].tolist()


In [ ]:
uk_reliability

In [27]:

stoplist = set(stopwords.words('english'))
punctuation = set(string.punctuation)
ps = PorterStemmer()

def preprocessing(list):
    preprocessed_comments = []
    for comment in list:
        comment = comment.lower()
        comment = ''.join([c for c in comment if c not in punctuation])
        
        tokens = word_tokenize(comment)
        #tokens = [token for token in tokens if token not in stoplist and token not in punctuation]
        tokens = [token for token in tokens if token not in stoplist and token not in punctuation and not token.isnumeric()]
        #tokens = [token for token in tokens if token.isnumeric()]
        tokens = [ps.stem(token) for token in tokens]
        
        for token in tokens:
            if token.isnumeric() == True:
                tokens.remove(token)
        
        preprocessed_comment = ' '.join(tokens)
        preprocessed_comments.append(preprocessed_comment)
    
    return preprocessed_comments

In [43]:
stoplist = set(stopwords.words('english'))
punctuation = string.punctuation + '‘’“”' 
ps = PorterStemmer()

def preprocessed_comments(comment_list):
    preprocessed_comments = []
    for comment in comment_list:
        comment = comment.lower()
        comment = ''.join([c for c in comment if c not in punctuation])
        tokens = word_tokenize(comment)
        tokens = [token for token in tokens if token not in stoplist and token not in punctuation and not token.isnumeric()]
        tokens = [ps.stem(token) for token in tokens]
        tokens = [token for token in tokens if token not in stoplist and token not in punctuation and not token.isnumeric()]
        preprocessed_comments.append(tokens)
    return preprocessed_comments

In [ ]:
print(preprocessed_comments(uk_speed))

In [ ]:
print(preprocessed_comments(uk_reliability))

In [ ]:
# finish with the preprocessing function, work on removing numeric values
# move on with next steps, making bowvector to be able to feed it into lda model 

In [46]:
# create a dictionary, each token is given an integer ID
uk_speed_preprocessed = preprocessed_comments(uk_speed)
speed_dictionary = corpora.Dictionary(preprocessed_comments(uk_speed))

uk_reliability_preprocessed = preprocessed_comments(uk_reliability)
reliability_dictionary = corpora.Dictionary(preprocessed_comments(uk_reliability))

print(reliability_dictionary)

Dictionary(2546 unique tokens: ['crucial', 'downtim', 'expens', 'hour', 'internet']...)


In [47]:
# convert tokens into BoW representation

speed_corpus = [speed_dictionary.doc2bow(comment) for comment in uk_speed_preprocessed]
reliability_corpus = [reliability_dictionary.doc2bow(comment) for comment in uk_reliability_preprocessed]

In [48]:
#train lda model
# set identified topics to a desired nr
# show 10 most important words (represented by probability of word belonging to topic) 
# assign label to topic based on most represented words

num_topics = 5
passes = 10
speed_lda = models.LdaModel(speed_corpus, id2word=speed_dictionary, num_topics=num_topics, passes=passes)
reliability_lda = models.LdaModel(reliability_corpus, id2word=reliability_dictionary, num_topics=num_topics, passes=passes)

In [ ]:
speed_lda.print_topics()
reliability_lda.print_topics()

In [ ]:
for topic in speed_lda.show_topics(num_topics=5, num_words=10, formatted=False):
    print('Topic {}:'.format(topic[0]))
    print([word[0] for word in topic[1]])
    print()

In [ ]:
#topic 0: internet speed & customer service, linking to virgin
#topic 1: broadband speed drops & customer service
#topic 2: broadband & internet, focus on price
#topic 3: slow internet, issues with internet & broadband connectivity, pay
#topic 4: slow wifi & broadband, poor connection, drops at times and doesnt always work

In [ ]:
for topic in reliability_lda.show_topics(num_topics=5, num_words=10, formatted=False):
    print('Topic {}:'.format(topic[0]))
    print([word[0] for word in topic[1]])
    print()

In [ ]:
#topic 0: at home internet drops
#topic 1: router, customer service, expensive
#topic 2: related to the company, calling customer service (?), connectivity issues
#topic 3: issues with pricing and wifi/broadband
#topic 4: wifi/broadband, customer service calling on the phone(?)

# CH

In [64]:
def no_stemming_preprocessed_comments(comment_list):
    preprocessed_comments = []
    for comment in comment_list:
        comment = comment.lower()
        comment = ''.join([c for c in comment if c not in punctuation])
        tokens = word_tokenize(comment)
        tokens = [token for token in tokens if token not in stoplist and token not in punctuation and not token.isnumeric()]
        #tokens = [ps.stem(token) for token in tokens]
        #tokens = [token for token in tokens if token not in stoplist and token not in punctuation and not token.isnumeric()]
        preprocessed_comments.append(tokens)
    return preprocessed_comments

In [97]:
ch_outages = pd.read_excel(r'C:CH(internet-outages).xlsx')
ch_reliability = pd.read_excel(r'C:CH(internet-reliability).xlsx')

In [98]:
ch_outages = ch_outages['comment'].tolist()
ch_reliability = ch_reliability['comment'].tolist()

In [99]:
ch_outages_preprocessed = no_stemming_preprocessed_comments(ch_outages)
ch_reliability_preprocessed = no_stemming_preprocessed_comments(ch_reliability)

In [ ]:
print(ch_reliability_preprocessed)

In [101]:
ch_outages_dictionary = corpora.Dictionary(ch_outages_preprocessed)
ch_reliability_dictionary = corpora.Dictionary(ch_reliability_preprocessed)

In [102]:
ch_outages_corpus = [ch_outages_dictionary.doc2bow(comment) for comment in ch_outages_preprocessed]
ch_reliability_corpus = [ch_reliability_dictionary.doc2bow(comment) for comment in ch_reliability_preprocessed]

In [103]:
num_topics = 4
passes = 10
ch_outages_lda = models.LdaModel(ch_outages_corpus, id2word=speed_dictionary, num_topics=num_topics, passes=passes)
ch_reliability_lda = models.LdaModel(ch_reliability_corpus, id2word=reliability_dictionary, num_topics=num_topics, passes=passes)

In [ ]:
for topic in ch_outages_lda.show_topics(num_topics=5, num_words=10, formatted=False):
    print('Topic {}:'.format(topic[0]))
    print([word[0] for word in topic[1]])
    print()

In [ ]:
for topic in ch_reliability_lda.show_topics(num_topics=5, num_words=10, formatted=False):
    print('Topic {}:'.format(topic[0]))
    print([word[0] for word in topic[1]])
    print()